In [1]:
import requests
import pandas as pd
from pandas_datareader import wb
#import the data_tools needed
import sys
sys.path.insert(0, "../../../data_tools/")
from mongo_tools import get_mongo_database,mongo_coll_to_dicts, mongo_to_dataframe, dataframe_to_mongo

#import the project specific data params
sys.path.insert(0, "../../")
from data_params import DATABASE, COLLECTIONS

In [7]:
OECD_ROOT_URL = 'http://stats.oecd.org/sdmx-json/data'
def make_OECD_request(dsname, dimensions, params=None, \
root_dir=OECD_ROOT_URL):
    """ Make a URL for the OECD API and return a response """

    if not params: 
        params = {}
    dim_args = ['+'.join(d) for d in dimensions] 
    dim_str = '.'.join(dim_args)

    url = root_dir + '/' + dsname + '/' + dim_str + '/all'
    print('Requesting URL: ' + url)
    return requests.get(url, params=params)

In [7]:
#World bank ==> use pandas_datareader wb
#Search WB indicators
#wb.search('population.*census').iloc[:,:2]
indicator = ['NY.GDP.MKTP.CD','SP.POP.TOTL']
col_rename = {
    'NY.GDP.MKTP.CD':'GDP_cst_dollars',
    'SP.POP.TOTL':'population'
}
country = ['all']
start = 1960
end = 2012

In [3]:
df = wb.download(indicator=indicator, country=country, start=start, end=end)
df = df.reset_index()
df.rename(columns=col_rename, inplace=True)
dataframe_to_mongo(df, DATABASE,'demo_eco_wb')

Starting new HTTP connection (1): api.worldbank.org
"GET /countries/all/indicators/NY.GDP.MKTP.CD?date=1960%3A2012&per_page=25000&format=json HTTP/1.1" 200 2271834
"GET /countries/all/indicators/SP.POP.TOTL?date=1960%3A2012&per_page=25000&format=json HTTP/1.1" 200 2162883


In [9]:
df.head()

,index,country,year,GDP_cst_dollars,population
0,0,Arab World,2012,2.773270e+12,368802611.0
1,1,Arab World,2011,2.497945e+12,361031820.0
2,2,Arab World,2010,2.103825e+12,353112237.0
3,3,Arab World,2009,1.798878e+12,345054176.0
4,4,Arab World,2008,2.081343e+12,336886468.0


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused